### Optimization Problem returns different solution everytime so re-running this code will result in different results than listed in paper and loaded into the dashboard 

**This notebook requires "pulp" package**

In [1]:
!pip install pulp --user

In [2]:
from pulp import *
import pandas as pd
from tqdm import tqdm
import random
import numpy as np

import warnings
warnings.filterwarnings("ignore")
random.seed(51)
np.random.seed(51)

In [3]:
def conv(string):
    return float(string.replace("%", ""))/100

loan_list = pd.read_csv('../data/test/X_test_y_pred.csv', 
                        usecols = ["Unnamed: 0.1", "loan_amnt", "int_rate", "grade", "score"],
                        converters = {'int_rate':conv},
                        dtype={'Unnamed: 0':'string', 'loan_amnt':'float'})

In [4]:
loan_list

,Unnamed: 0.1,loan_amnt,int_rate,grade,score
0,1238363,4200.0,0.1695,C,0.371431
1,1238336,6700.0,0.2565,D,0.232696
2,1238316,6000.0,0.1524,C,0.503907
3,1238195,40000.0,0.0646,A,0.145252
4,1238087,3000.0,0.2055,D,0.096426
...,...,...,...,...,...
9995,1850384,10000.0,0.1824,C,0.128284
9996,1850339,3000.0,0.2074,C,0.205255
9997,1851065,3200.0,0.0956,A,0.115259
9998,1851076,3500.0,0.2074,C,0.416174


In [5]:
# Get a list of ids
ids = list(loan_list['Unnamed: 0.1'])

In [6]:
# Initialize Dictionaries for Loans, Interest Rates, Grades, and Default Probability for each loan id
# loans = dict(zip(ids, loan_list['loan_amnt']))
int_rates = dict(zip(ids, loan_list['int_rate']))
grades = dict(zip(ids, loan_list['grade']))
default_prob = dict(zip(ids, loan_list['score']))

In [7]:
# Get ids for each grade
A_grade = [i for i in grades.keys() if grades[i] == 'A']
B_grade = [i for i in grades.keys() if grades[i] == 'B']
C_grade = [i for i in grades.keys() if grades[i] == 'C']
D_grade = [i for i in grades.keys() if grades[i] == 'D']
E_grade = [i for i in grades.keys() if grades[i] == 'E']

In [8]:
# Set loan id to Take either 1 or 0 values (chosen or not)
id_vars = LpVariable.dicts("ID", ids, lowBound=0, upBound=1, cat='Integer')

In [9]:
# Set a dictionary of Grades
grade_combo = {"A":([1,0,0,0],[1,0,0,0]), 
               "B":([0,1,0,0],[0,1,0,0]),
               "C":([0,0,1,0],[0,0,1,0]),
               "D":([0,0,0,1],[0,0,0,1]),
               "AB":([.4,.4,0,0],[1,1,0,0]),
               "AC":([.4,0,.4,0],[1,0,1,0]),
               "AD":([.4,0,0,.4],[1,0,0,1]),
               "BC":([0,.4,.4,0],[0,1,1,0]),
               "BD":([0,.4,0,.4],[0,1,0,1]),
               "CD":([0,0,.4,.4],[0,0,1,1]),
               "ABC":([.25,.2,.25,0],[1,1,1,0]),
               "ABD":([.25,.2,0,.25],[1,1,0,1]),
               "ACD":([.25,0,.2,.25],[1,0,1,1]),
               "BCD":([0,.25,.2,.25],[0,1,1,1]),
               "ABCD":([.2,.2,.2,.2],[1,1,1,1])}

# grade_combo = {"ABCD":([.2,.2,.2,.2],[1,1,1,1])}

In [10]:
# Set a list of an exact loan quantity portfolio
qtty_portfolio = [5, 10, 15, 20]
# qtty_portfolio = [5]

In [11]:
# Initialize optimized portfolios table
df = pd.DataFrame(columns = ['id', 'max_qtty', 'grade'])

In [12]:
for q in qtty_portfolio:
    for idx in tqdm(grade_combo):

        # Setup the problem
        expected_return = LpProblem("Best_Expected_Return_Problem", LpMaximize)

        # Set the objective function
        # expected_cash_flow += lpSum([loans[i] * pow(1 + int_rates[i]/365, 3*365) * (1 - default_prob[i]) * id_vars[i] for i in id_vars])
    
        expected_return += lpSum([(1+int_rates[i])*(1 - default_prob[i]) * id_vars[i] for i in id_vars])

        # Set quantity of loans
        expected_return += lpSum([id_vars[i] for i in id_vars]) == q

        # A-Grade loan amount
        expected_return += lpSum([id_vars[i] for i in A_grade]) >= grade_combo[idx][0][0] * lpSum([id_vars[i] for i in id_vars])
        expected_return += lpSum([id_vars[i] for i in A_grade]) <= grade_combo[idx][1][0] * lpSum([id_vars[i] for i in id_vars])

        # B-Grade loan amount
        expected_return += lpSum([id_vars[i] for i in B_grade]) >= grade_combo[idx][0][1] * lpSum([id_vars[i] for i in id_vars])
        expected_return += lpSum([id_vars[i] for i in B_grade]) <= grade_combo[idx][1][1] * lpSum([id_vars[i] for i in id_vars])

        # C-Grade loan amount
        expected_return += lpSum([id_vars[i] for i in C_grade]) >= grade_combo[idx][0][2] * lpSum([id_vars[i] for i in id_vars])
        expected_return += lpSum([id_vars[i] for i in C_grade]) <= grade_combo[idx][1][2] * lpSum([id_vars[i] for i in id_vars])

        # D-Grade loan amount
        expected_return += lpSum([id_vars[i] for i in D_grade]) >= grade_combo[idx][0][3] * lpSum([id_vars[i] for i in id_vars])
        expected_return += lpSum([id_vars[i] for i in D_grade]) <= grade_combo[idx][1][3] * lpSum([id_vars[i] for i in id_vars])

#         expected_return.solve(SCIP_CMD(timeLimit=1800, threads=None))
        expected_return.solve(PULP_CBC_CMD(timeLimit=1800))

        if expected_return.status == 1:
            for v in expected_return.variables():
                if v.varValue > 0:
                    df = df.append({'id':v.name.replace("ID_", ""), 'max_qtty':q, 'grade':idx}, ignore_index = True)

        del expected_return    

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.65s/it]


In [13]:
df.to_csv("../data/expected_return_porfolios.csv",index=False)

In [14]:
df[df['grade'] == "ABC"]

,id,max_qtty,grade
50,1393400,5,ABC
51,1833702,5,ABC
52,1841071,5,ABC
53,1861628,5,ABC
54,1867555,5,ABC
175,1393400,10,ABC
176,1464704,10,ABC
177,1474600,10,ABC
178,1827116,10,ABC
179,1828799,10,ABC
